# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 19 2022 10:41AM,253360,16,8921747,Sartorius Bioprocess Solutions,Released
1,Dec 19 2022 10:19AM,253358,15,8921193,"Brookfield Pharmaceuticals, LLC",Released
2,Dec 19 2022 10:19AM,253358,15,8921194,"Brookfield Pharmaceuticals, LLC",Released
3,Dec 19 2022 10:19AM,253358,15,8921195,"Brookfield Pharmaceuticals, LLC",Released
4,Dec 19 2022 10:19AM,253358,15,8921196,"Brookfield Pharmaceuticals, LLC",Released
5,Dec 19 2022 10:19AM,253358,15,8921197,"Brookfield Pharmaceuticals, LLC",Released
6,Dec 19 2022 10:19AM,253358,15,8921198,"Brookfield Pharmaceuticals, LLC",Released
7,Dec 19 2022 10:19AM,253358,15,8921199,"Brookfield Pharmaceuticals, LLC",Released
8,Dec 19 2022 10:19AM,253358,15,8921200,"Brookfield Pharmaceuticals, LLC",Released
9,Dec 19 2022 10:19AM,253358,15,8921201,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,253356,Released,21
17,253357,Released,1
18,253358,Released,22
19,253359,Released,23
20,253360,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253356,NaN,NaN,21.0
253357,NaN,NaN,1.0
253358,NaN,NaN,22.0
253359,NaN,NaN,23.0
253360,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1.0,1.0,11.0
253335,0.0,0.0,28.0
253337,0.0,1.0,0.0
253340,0.0,0.0,19.0
253341,0.0,5.0,34.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1,1,11
253335,0,0,28
253337,0,1,0
253340,0,0,19
253341,0,5,34


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,1,11
1,253335,0,0,28
2,253337,0,1,0
3,253340,0,0,19
4,253341,0,5,34


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,1,11
1,253335,,,28
2,253337,,1,
3,253340,,,19
4,253341,,5,34


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 19 2022 10:41AM,253360,16,Sartorius Bioprocess Solutions
1,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC"
23,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation
85,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC"
108,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation
148,Dec 19 2022 10:12AM,253351,10,ISDIN Corporation
183,Dec 19 2022 10:12AM,253348,10,ISDIN Corporation
237,Dec 19 2022 10:11AM,253356,10,ISDIN Corporation
258,Dec 19 2022 10:08AM,253357,10,"ClearSpec, LLC"
259,Dec 19 2022 10:03AM,253346,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 19 2022 10:41AM,253360,16,Sartorius Bioprocess Solutions,,,1
1,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",,,22
2,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,,,62
3,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",,,23
4,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,,,40
5,Dec 19 2022 10:12AM,253351,10,ISDIN Corporation,,,35
6,Dec 19 2022 10:12AM,253348,10,ISDIN Corporation,,,54
7,Dec 19 2022 10:11AM,253356,10,ISDIN Corporation,,,21
8,Dec 19 2022 10:08AM,253357,10,"ClearSpec, LLC",,,1
9,Dec 19 2022 10:03AM,253346,15,"Alliance Pharma, Inc.",,5,30


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 10:41AM,253360,16,Sartorius Bioprocess Solutions,1,,
1,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,,
2,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,62,,
3,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",23,,
4,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,40,,
5,Dec 19 2022 10:12AM,253351,10,ISDIN Corporation,35,,
6,Dec 19 2022 10:12AM,253348,10,ISDIN Corporation,54,,
7,Dec 19 2022 10:11AM,253356,10,ISDIN Corporation,21,,
8,Dec 19 2022 10:08AM,253357,10,"ClearSpec, LLC",1,,
9,Dec 19 2022 10:03AM,253346,15,"Alliance Pharma, Inc.",30,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 10:41AM,253360,16,Sartorius Bioprocess Solutions,1,,
1,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,,
2,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,62,,
3,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",23,,
4,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,40,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 10:41AM,253360,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22.0,NaN,NaN
2,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,62.0,NaN,NaN
3,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",23.0,NaN,NaN
4,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,40.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 10:41AM,253360,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22.0,0.0,0.0
2,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,62.0,0.0,0.0
3,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",23.0,0.0,0.0
4,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,40.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2280137,272.0,5.0,0.0
12,253335,28.0,0.0,0.0
15,760063,75.0,5.0,0.0
16,760040,1.0,2.0,0.0
19,253332,11.0,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2280137,272.0,5.0,0.0
1,12,253335,28.0,0.0,0.0
2,15,760063,75.0,5.0,0.0
3,16,760040,1.0,2.0,0.0
4,19,253332,11.0,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,272.0,5.0,0.0
1,12,28.0,0.0,0.0
2,15,75.0,5.0,0.0
3,16,1.0,2.0,0.0
4,19,11.0,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,272.0
1,12,Released,28.0
2,15,Released,75.0
3,16,Released,1.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19
Status,,,,,
Completed,0.0,0.0,0.0,0.0,1.0
Executing,5.0,0.0,5.0,2.0,1.0
Released,272.0,28.0,75.0,1.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19
0,Completed,0.0,0.0,0.0,0.0,1.0
1,Executing,5.0,0.0,5.0,2.0,1.0
2,Released,272.0,28.0,75.0,1.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19
0,Completed,0.0,0.0,0.0,0.0,1.0
1,Executing,5.0,0.0,5.0,2.0,1.0
2,Released,272.0,28.0,75.0,1.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()